In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q tensorflow
!pip install -q tensorflow_ranking

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os

PATH = os.getcwd()
#PATH = '/content/drive/Shareddrives/Master Tesis/Tesis'

In [11]:
import tensorflow_ranking as tfr
import tensorflow as tf
from tensorflow_serving.apis import input_pb2

final_table = pd.read_csv(os.path.join(PATH, 'Tables', 'final_table.csv'))
final_table['Query'] = pd.to_datetime(final_table['Query']).dt.date
final_table

,Query,Ticker,big_log_ret,big_RCV,big_RVT,big_positivePartscr,big_negativePartscr,big_splogscr,big_linscr
0,2012-01-08,AAL,0.063980,8.476000,0.000280,0.020240,0.013340,0.010760,54.733580
1,2012-01-08,AAPL,-0.006151,13.162167,0.003400,0.016050,0.015650,0.020333,48.380133
2,2012-01-08,ABC,-0.020684,0.404000,0.000760,0.027820,0.012980,0.017000,65.547120
3,2012-01-08,ABT,0.000863,-24.607200,0.000320,0.010820,0.018540,-0.012980,27.375300
4,2012-01-08,AMZN,-0.023212,-3.370000,0.001100,0.016180,0.020000,-0.024640,40.688020
...,...,...,...,...,...,...,...,...,...
41819,2021-11-28,UPS,-0.022512,-42.160333,0.000317,0.018200,0.024717,-0.008817,46.677617
41820,2021-11-28,USB,-0.033782,-28.514833,0.000483,0.020517,0.010367,0.037033,68.229333
41821,2021-11-28,VZ,-0.007363,-22.016000,0.000686,0.018957,0.036629,-0.066443,28.958114
41822,2021-11-28,WFC,-0.014140,-37.779000,0.000983,0.004283,0.017333,-0.061550,15.077883


In [ ]:
# Store the paths to files containing training and test instances.
_TRAIN_DATA_PATH = "./Dataset-tfrecords/Split_1/train.tfrecord"
#_TEST_DATA_PATH =  f"{dataset_folder_path}/test-fold1.gzip_tfrecord"

# The maximum number of documents per query in the dataset.
# Document lists are padded or truncated to this size.
_LIST_SIZE = 200

# The document relevance label.
_LABEL_FEATURE_NAME = "rel"
_NUM_FEATURES = final_table.shape[1] -3

# Padding labels are set negative so that the corresponding examples can be
# ignored in loss and metrics.
_PADDING_LABEL = -1

# Learning rate for optimizer.
_LEARNING_RATE = 0.05

# Parameters to the scoring function.
_BATCH_SIZE = 32
_DROPOUT_RATE = 0.5

# Location of model directory and number of training steps.
_MODEL_DIR = f"./model_logs/model_{datetime.datetime.now().strftime('%m-%d-%Y_%H-%M-%S')}"

# setting as shell env for tensorboard stuff
os.environ["models_dir"] = _MODEL_DIR

In [ ]:
def create_feature_columns():
    # We dont have context features in our datasets
    context_feature_columns = {}
    feature_names = ["{}".format(i + 1) for i in range(_NUM_FEATURES)]
    example_feature_columns = {
        name:
        tf.feature_column.numeric_column(name, shape=(1,), default_value=0.0)
        for name in feature_names}
    
    return context_feature_columns, example_feature_columns

In [ ]:
def create_dataset_from_tfrecords(input_path:str,
                                  batch_sz:int,
                                  shuffle:bool = True,
                                  num_epochs:int = None,
                                  data_format:str = "ELWC",
                                  compression_type:str = ''):

    context_feature_columns, example_feature_columns = create_feature_columns()


    context_feature_spec = tf.feature_column.make_parse_example_spec(
      context_feature_columns.values())
    label_column = tf.feature_column.numeric_column(
      _LABEL_FEATURE_NAME, dtype=tf.int64, default_value=_PADDING_LABEL)
    example_feature_spec = tf.feature_column.make_parse_example_spec(
      list(example_feature_columns.values()) + [label_column])

    _reader_arg_list = []
    if compression_type:
        assert compression_type in ["", "GZIP","ZLIB"]
        _reader_arg_list = [compression_type]


    dataset = tfr.data.build_ranking_dataset(
      file_pattern=input_path,
      data_format=tfr.data.ELWC,
      batch_size=batch_sz,
      list_size=_LIST_SIZE,
      context_feature_spec=context_feature_spec,
      example_feature_spec=example_feature_spec,
      reader=tf.data.TFRecordDataset,
      reader_args= _reader_arg_list,
      shuffle=shuffle,
      num_epochs=num_epochs,
      )

    def _log1p_transform(features):
        '''
        computes elementwise log_e(|x|)*sign(x)
        '''
    transformed_feats = {
        f:tf.math.multiply(
            tf.math.log1p(
                tf.math.abs(features[f])
                ),
            tf.math.sign(features[f])
            )
        for f in features}
    return transformed_feats

    def _split_label_and_transform_features(features):
        label = tf.squeeze(features.pop(_LABEL_FEATURE_NAME), axis=2)
        label = tf.cast(label, tf.float32)
        features = _log1p_transform(features)

    return features, label

    dataset = dataset.map(_split_label_and_transform_features)

    return dataset

In [ ]:
context_feature_columns, example_feature_columns = create_feature_columns()
# Using a Canned Network
ranking_network = tfr.keras.canned.DNNRankingNetwork(
      context_feature_columns=context_feature_columns,
      example_feature_columns=example_feature_columns,
      hidden_layer_dims=[64, 32, 16],
      activation=tf.nn.relu,
      dropout=_DROPOUT_RATE,
      use_batch_norm=True,
      batch_norm_moment=0.4)


In [ ]:
_loss_obj = tfr.keras.losses.get(
    tfr.losses.RankingLossKey.GUMBEL_APPROX_NDCG_LOSS)


#Different Losses knowing that the softmax loss is representative of the listwise apporach
#_loss_obj = tfr.keras.losses.get(
#    tfr.losses.RankingLossKey.UNIQUE_SOFTMAX_LOSS)


#_loss_obj = tfr.keras.losses.get(
#    tfr.losses.RankingLossKey.LIST_MLE_LOSS)


#_loss_obj = tfr.keras.losses.get(
#    tfr.losses.RankingLossKey.SOFTMAX_LOSS)
# Contains all ranking metrics, including NDCG @ {1, 3, 5, 10}.

def _make_eval_metric_fns():
  """Returns a list of ranking metrics for the keras ranker"""
  metric_fns = [tfr.keras.metrics.get(**kwargs) 
                        for kwargs in [dict(key="ndcg", topn=topn, 
                                        name="metric/ndcg_{}".format(topn)) 
                                            for topn in [1, 3, 5, 10]]
                ]
  return metric_fns

default_metrics = _make_eval_metric_fns()

config = tf.estimator.RunConfig(
      model_dir=_MODEL_DIR,
      keep_checkpoint_max=10,
      save_checkpoints_secs=200)

In [ ]:
# Build ranker as a Functional Keras model.
ranker = tfr.keras.model.create_keras_model(
      network=ranking_network,
      loss=_loss_obj,
      metrics=default_metrics,
      optimizer=tf.keras.optimizers.Adagrad(learning_rate=_LEARNING_RATE),
      size_feature_name=None)


In [ ]:
assert tf.test.gpu_device_name() != '', "GPU not detected, training is much faster GPU/TPU instance of colab"

train_dataset = create_dataset_from_tfrecords(_TRAIN_DATA_PATH,
                                              _BATCH_SIZE,
                                              compression_type="GZIP")

vali_dataset = create_dataset_from_tfrecords(_VALID_DATA_PATH,
                                             _BATCH_SIZE,
                                             shuffle=False,
                                             num_epochs=1, 
                                             compression_type="GZIP")

tensorboard_callback = tf.keras.callbacks.TensorBoard(_MODEL_DIR)


ranker.fit(train_dataset,
           validation_data=vali_dataset,
           steps_per_epoch=100,
           epochs=100,
           validation_steps=1,
           callbacks=[tensorboard_callback])
